In [66]:
import GPy
import string
import numpy as np
from torch import nn
import pandas as pd
from sklearn.decomposition import PCA
#import cPickle as pickle
from GPy.util import pca
from matplotlib import pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import optim
from sklearn import svm
from scipy.linalg import sqrtm
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import FactorAnalysis
#from sklearn.utils import resample
%matplotlib inline

In [67]:
def get_data(data_type,file_num):

    if(data_type=='train'):
        data=pd.read_csv(f"Dataset/IHDP_a/ihdp_npci_train_{file_num}.csv")
        #data=pd.read_csv(f"Data/ihdp_npci_train_{file_num}.csv",index_col=False)
    else:
        data = pd.read_csv(f"Dataset/IHDP_a/ihdp_npci_test_{file_num}.csv")
        #data = pd.read_csv(f"Data/ihdp_npci_test_{file_num}.csv",index_col=False)

    x_data=pd.concat([data.iloc[:,0], data.iloc[:, 1:30]], axis = 1)
 
   
    y_data=data.iloc[:, 1]
    
    return x_data,y_data

def cal_pehe(y_1,y_0,test_x):
    #pehe...
    cate_pred=y_1-y_0
    #cate_pred=y1-y0
    cate_true=test_x.iloc[:,4]-test_x.iloc[:,3] #Hill's noiseless true values
    cate_err=np.mean( np.square( ( (cate_true) - (cate_pred) ) ) )
    return np.sqrt(cate_err)

def cal_pehe_2(data,model):
    #data,y=get_data('test',i)

    data=data.to_numpy()
    data=torch.from_numpy(data.astype(np.float32))



    concat_pred=model(data[:,5:30])
    #print(concat_pred)
    #dont forget to rescale the outcome before estimation!
    #y0_pred = data['y_scaler'].inverse_transform(concat_pred[:, 0].reshape(-1, 1))
    #y1_pred = data['y_scaler'].inverse_transform(concat_pred[:, 1].reshape(-1, 1))
    cate_pred=concat_pred[:,1]-concat_pred[:,0]
    cate_true=data[:,4]-data[:,3] #Hill's noiseless true values


    cate_err=torch.mean( torch.square( ( (cate_true) - (cate_pred) ) ) )

    return torch.sqrt(cate_err).item()

In [68]:
def train_GPLVM(input_dim,Yn,kr):
    input_dim = input_dim
    # define covariance function to be used
    if(kr=='RBF'):
        kernel = GPy.kern.RBF(input_dim, ARD=True)
    else:
        kernel = GPy.kern.Linear(input_dim, ARD=True)
        
    # setup the GPLVM model
    m = GPy.models.GPLVM(Yn, input_dim=input_dim, kernel=kernel)
    # optimize for 1000 iterations
    m.optimize(messages=1, max_iters=1000) 
    return m

In [69]:
def upsample(x_data, y_data):
    x_train_sr=x_data[x_data['treatment']==0]
    y_train_sr=y_data[x_data['treatment']==0]
    x_train_tr=x_data[x_data['treatment']==1]
    y_train_tr=y_data[x_data['treatment']==1]
    
    
    return x_train_sr,y_train_sr,x_train_tr,y_train_tr
    

def SA(sr,tr,lat_dim):
    pca_src_ = PCA(lat_dim)
    pca_tgt_ = PCA(lat_dim)
    pca_src_.fit(sr)
    pca_tgt_.fit(tr)
    Xs=pca_src_.components_.transpose()
    Xt=pca_tgt_.components_.transpose()
    
    sr_aligned=sr@(Xs @ Xs.transpose()@Xt);
    tr_projected = tr@Xt;
    #tr_projected = tr@(Xt @ Xt.transpose()@Xs);
    
    
    return sr_aligned, tr_projected,pca_src_,pca_tgt_

def SA_FA(sr,tr,lat_dim):
    
    pca_src_  = FactorAnalysis(n_components=lat_dim, random_state=0)
    pca_tgt_ = FactorAnalysis(n_components=lat_dim, random_state=0)
    #pca_src_ = PCA(lat_dim)
    #pca_tgt_ = PCA(lat_dim)
    pca_src_.fit(sr)
    pca_tgt_.fit(tr)
    Xs=pca_src_.components_.transpose()
    Xt=pca_tgt_.components_.transpose()
    
    sr_aligned=sr@(Xs @ Xs.transpose()@Xt);
    tr_projected = tr@Xt;
    #tr_projected = tr@(Xt @ Xt.transpose()@Xs);
    
    
    return sr_aligned, tr_projected,pca_src_,pca_tgt_

def coral(Xs,Xt,lamda_):
    
  
        cov_Xs = np.cov(Xs,rowvar=False)
        cov_Xt = np.cov(Xt,rowvar=False)
        
          
        Cs_ = cov_Xs + lambda_ * np.eye(Xs.shape[1])
        Ct_ = cov_Xt + lambda_ * np.eye(Xt.shape[1])
        
        Cs_sqrt_inv = np.linalg.inv(sqrtm(Cs_))
        Ct_sqrt = sqrtm(Ct_)
        
        if np.iscomplexobj(Cs_sqrt_inv):
            Cs_sqrt_inv = Cs_sqrt_inv.real
        if np.iscomplexobj(Ct_sqrt):
            Ct_sqrt = Ct_sqrt.real
        
        Xs_emb = np.matmul(Xs, Cs_sqrt_inv)
        Xs_emb = np.matmul(Xs_emb, Ct_sqrt)
        
        return Xs_emb,Xt
    
    

def coral_FA(Xs,Xt,lamda_,dim):
    
    
        pca_src_  = FactorAnalysis(n_components=dim, random_state=0)
        pca_tgt_ = FactorAnalysis(n_components=dim, random_state=0)
        pca_src_.fit(Xs)
        pca_tgt_.fit(Xt)
        cov_Xs=pca_src_.get_covariance()
        cov_Xt=pca_tgt_.get_covariance()
        #cov_Xs = np.cov(Xs,rowvar=False)
        #cov_Xt = np.cov(Xt,rowvar=False)
        
          
        Cs_ = cov_Xs + lambda_ * np.eye(Xs.shape[1])
        Ct_ = cov_Xt + lambda_ * np.eye(Xt.shape[1])
        
        Cs_sqrt_inv = np.linalg.inv(sqrtm(Cs_))
        Ct_sqrt = sqrtm(Ct_)
        
        if np.iscomplexobj(Cs_sqrt_inv):
            Cs_sqrt_inv = Cs_sqrt_inv.real
        if np.iscomplexobj(Ct_sqrt):
            Ct_sqrt = Ct_sqrt.real
        
        Xs_emb = np.matmul(Xs, Cs_sqrt_inv)
        Xs_emb = np.matmul(Xs_emb, Ct_sqrt)
        
        return Xs_emb,Xt
    
    
def coral_GPLVM(Xs,Xt,lamda_, lat_dim,kr):
    
       # if (train==True):
            
        
        msr=train_GPLVM(lat_dim,Xs.to_numpy(),kr)
        mtr=train_GPLVM(lat_dim,Xt.to_numpy(),kr)

        Xs=msr.X
        Xt=mtr.X
        cov_Xs = np.cov(Xs,rowvar=False)
        cov_Xt = np.cov(Xt,rowvar=False)
        
        Cs_ = cov_Xs + lambda_ * np.eye(Xs.shape[1])
        Ct_ = cov_Xt + lambda_ * np.eye(Xt.shape[1])
        
        Cs_sqrt_inv = np.linalg.inv(sqrtm(Cs_))
        Ct_sqrt = sqrtm(Ct_)
        
        if np.iscomplexobj(Cs_sqrt_inv):
            Cs_sqrt_inv = Cs_sqrt_inv.real
        if np.iscomplexobj(Ct_sqrt):
            Ct_sqrt = Ct_sqrt.real
        
        Xs_emb = np.matmul(Xs, Cs_sqrt_inv)
        Xs_emb = np.matmul(Xs_emb, Ct_sqrt)
        
        return Xs_emb,Xt
    
    
def coral_GPLVM_FA(Xs,Xt,lamda_, lat_dim,kr):
    
       # if (train==True):
          
            
        pca_src_  = FactorAnalysis(n_components=lat_dim, random_state=0)
        pca_tgt_ = FactorAnalysis(n_components=lat_dim, random_state=0)
        
        Xs=pca_src_.fit(Xs)
        Xt=pca_tgt_.fit(Xt)
        
        Xs=pca_src_.transform(Xs)
        Xt=pca_tgt_.transform(Xt)
        
        #msr=train_GPLVM(lat_dim,Xs.to_numpy(),kr)
        #mtr=train_GPLVM(lat_dim,Xt.to_numpy(),kr)

        #Xs=msr.X
        #Xt=mtr.X
        
        cov_Xs = np.cov(Xs,rowvar=False)
        cov_Xt = np.cov(Xt,rowvar=False)
        
        Cs_ = cov_Xs + lambda_ * np.eye(Xs.shape[1])
        Ct_ = cov_Xt + lambda_ * np.eye(Xt.shape[1])
        
        Cs_sqrt_inv = np.linalg.inv(sqrtm(Cs_))
        Ct_sqrt = sqrtm(Ct_)
        
        if np.iscomplexobj(Cs_sqrt_inv):
            Cs_sqrt_inv = Cs_sqrt_inv.real
        if np.iscomplexobj(Ct_sqrt):
            Ct_sqrt = Ct_sqrt.real
        
        Xs_emb = np.matmul(Xs, Cs_sqrt_inv)
        Xs_emb = np.matmul(Xs_emb, Ct_sqrt)
        
        return Xs_emb,Xt

In [70]:
#calling SA
#fil_num=50
input_dim=20
kr='Linear' #RBF
pehe_error=[]
reg_0 = svm.SVR()
reg_1=svm.SVR()

for i in range(1,101):
    x_data,y_data=get_data('train',i)
    test_x,test_y=get_data('test',i)
    x_sr,y_sr,x_tr,y_tr=upsample(x_data, y_data)
    
    #x_sr_tran=x_sr.iloc[:,5:30].to_numpy().transpose()
    #x_tr_tran=x_tr.iloc[:,5:30].to_numpy().transpose()
    #sr_aligned,emd_tr,msr,mtr=SA(x_sr_tran,x_tr_tran,input_dim,kr)
    sr,tr,pca_sr,pca_tr=SA(x_sr.iloc[:,5:30],x_tr.iloc[:,5:30],input_dim)
    
    # Test data transformation
    test_sr=test_x[test_x['treatment']==0]
    #test_sr_y=test_y[test_x['treatment']==1]
    test_tr=test_x[test_x['treatment']==1]
    #test_tr_y=test_y[test_x['treatment']==0]
    
    sr_test=test_sr.iloc[:,5:30].to_numpy()
    tr_test=test_tr.iloc[:,5:30].to_numpy()
    
    Xs_test=pca_sr.components_.transpose()
    Xt_test=pca_tr.components_.transpose()
    
    sr_aligned_test=sr_test@(Xs_test @ Xs_test.transpose()@Xt_test);
    tr_projected_test= tr_test@Xt_test;
    #tr_projected_test= tr_test@(Xt_test @ Xt_test.transpose()@Xs_test);
    
    
    test_complt=pd.concat([test_sr.iloc[:,0:5],test_tr.iloc[:,0:5]], axis=0)
    test_transformed_complt=np.concatenate((sr_aligned_test,tr_projected_test), axis=0)
    
    # Selectio bias measure
    trans_comp=np.concatenate((sr,tr), axis=0)
    t_comp=np.concatenate((x_sr.iloc[:,0],x_tr.iloc[:,0]), axis=0)
    clf = LogisticRegression(random_state=0).fit(trans_comp, t_comp)
    #p=clf.predict_proba(test_transformed_complt)
    p=clf.predict_proba(trans_comp)
    
    
    reg_0.fit(sr.values, y_sr.values)
    y_0=reg_0.predict(test_transformed_complt)
    reg_1.fit(tr.values, y_tr.values)
    y_1=reg_1.predict(test_transformed_complt)
    pehe=cal_pehe(y_1,y_0,test_complt)
    pehe_error.append(pehe)

In [71]:
np.mean(pehe_error)

1.1300980368868832

In [72]:
np.std(pehe_error)

1.0324575950071806

In [73]:
true_prob=[1-np.array(t_comp).mean(), np.array(t_comp).mean()]

In [74]:
np.sqrt(((p[:,1] - true_prob[1])**2).mean())

0.34976332912943686

In [76]:
#calling coral/coral_GPLVM
#fil_num=50
input_dim=25
kr='Linear' #RBF
pehe_error=[]
lambda_=1
batch_size=32
epochs=300
reg_0 = svm.SVR()#MLPRegressor(random_state=1, max_iter=1000,shuffle=False, solver='adam',learning_rate_init=0.01,early_stopping=True,hidden_layer_sizes=(100,100))
reg_1=svm.SVR()#MLPRegressor(random_state=1, max_iter=1000,shuffle=False,solver='adam',learning_rate_init=0.01,early_stopping=True,hidden_layer_sizes=(100,100))#svm.SVR()

for i in range(1,101):
    x_data,y_data=get_data('train',i)
    test_x,test_y=get_data('test',i)
    x_sr,y_sr,x_tr,y_tr=upsample(x_data, y_data)
    
    #x_sr_tran=x_sr.iloc[:,5:30].to_numpy().transpose()
    #x_tr_tran=x_tr.iloc[:,5:30].to_numpy().transpose()
    #sr_aligned,emd_tr,msr,mtr=SA(x_sr_tran,x_tr_tran,input_dim,kr)
    sr,tr=coral(x_sr.iloc[:,5:30],x_tr.iloc[:,5:30],lambda_)
    #sr,tr=coral_GPLVM_FA(x_sr.iloc[:,5:30],x_tr.iloc[:,5:30],lambda_,input_dim,kr)
    
    # Test data transformation
    test_sr=test_x[test_x['treatment']==0]
    #test_sr_y=test_y[test_x['treatment']==1]
    test_tr=test_x[test_x['treatment']==1]
    #test_tr_y=test_y[test_x['treatment']==0]
    
    sr_test=test_sr.iloc[:,5:30]
    tr_test=test_tr.iloc[:,5:30]
    
    
    # Test Transformation
    
    sr_t, tr_t=coral(sr_test,tr_test,lambda_)
    #sr_t, tr_t=coral_GPLVM_FA(sr_test,tr_test,lambda_,input_dim,kr)
    
    test_complt=pd.concat([test_sr.iloc[:,0:5],test_tr.iloc[:,0:5]], axis=0)
    test_transformed_complt=np.concatenate((sr_t,tr_t), axis=0)
    trans_comp=np.concatenate((sr,tr), axis=0)
    untransformed_data=np.concatenate((x_sr.iloc[:,5:30],x_tr.iloc[:,5:30]), axis=0)
    t_comp=np.concatenate((x_sr.iloc[:,0],x_tr.iloc[:,0]), axis=0)
    
    
    # train NN for Regression task
    # combine train data
    """
    s_train_data=pd.concat([x_sr.iloc[:,0:5],sr],axis=1)
    t_train_data=pd.concat([x_tr.iloc[:,0:5],tr],axis=1)
    com_x=pd.concat([s_train_data,t_train_data],axis=0)
    com_y=pd.concat([y_sr,y_tr],axis=0)
    model=train_Regressor(com_x,com_y,batch_size,epochs)
    test_data=pd.concat([test_complt,test_transformed_complt],axis=1)
    pehe=cal_pehe_2(test_data,model)
    """


    reg_0.fit(sr.values, y_sr.values)
    y_0=reg_0.predict(test_transformed_complt)
    
    # calculation about about P(t|x)
    #trans_comp=np.concatenate((x_sr.iloc[:,5:30],x_tr.iloc[:,5:30]), axis=0)
    clf = LogisticRegression(random_state=0).fit(trans_comp, t_comp)
    
    #p=clf.predict_proba(test_transformed_complt)
    p=clf.predict_proba(trans_comp)
    
    reg_1.fit(tr.values, y_tr.values)
    y_1=reg_1.predict(test_transformed_complt)
    pehe=cal_pehe(y_1,y_0,test_complt)  
    pehe_error.append(pehe)

In [77]:
np.mean(pehe_error)

1.09799355646089

In [78]:
np.std(pehe_error)

1.003280612852779

In [79]:
true_prob=[1-np.array(t_comp).mean(), np.array(t_comp).mean()]

In [80]:
np.sqrt(((p[:,1] - true_prob[1])**2).mean())

0.15314371597193827

In [198]:
#t_sr_data=msr.infer_newX(test_sr.iloc[:,5:30].to_numpy())
#emd_sr_test=np.array(t_sr_data[0])
#t_tr_data=mtr.infer_newX(test_tr.iloc[:,5:30].to_numpy())
#emd_tr_test=np.array(t_tr_data[0])

#sr_aligned_test=(emd_sr_test @ emd_sr_test.transpose()@emd_tr_test);
#test_complt=pd.concat([test_sr.iloc[:,0:5],test_tr.iloc[:,0:5]], axis=0)
#test_embd_complt=np.concatenate((sr_aligned_test,emd_tr_test), axis=0)

#reg_1.fit(sr_aligned, y_sr)
#y_1=reg_1.predict(test_embd_complt)
#reg_0.fit(emd_tr, y_tr)
#y_0=reg_0.predict(test_embd_complt)
#pehe=cal_pehe(y_1,y_0,test_complt)
#pehe_error.append(pehe)

In [173]:
"""
# Principal Component Analysis
from numpy import array
from sklearn.decomposition import PCA
# define a matrix
A = array([[1, 2,7,11], [3, 4,8,12], [5, 6,9,13]])
print(A)
# create the PCA instance
pca = PCA(2)
# fit on data
pca.fit(A)
# access values and vectors
print(pca.components_)
#print(pca.explained_variance_)
# transform data
#B = pca.transform(A)
#print(B)

"""


'\n# Principal Component Analysis\nfrom numpy import array\nfrom sklearn.decomposition import PCA\n# define a matrix\nA = array([[1, 2,7,11], [3, 4,8,12], [5, 6,9,13]])\nprint(A)\n# create the PCA instance\npca = PCA(2)\n# fit on data\npca.fit(A)\n# access values and vectors\nprint(pca.components_)\n#print(pca.explained_variance_)\n# transform data\n#B = pca.transform(A)\n#print(B)\n\n'

In [174]:
"""
svd = np.linalg.svd
A = array([[1, 2,7,11], [3, 4,8,12], [5, 6,9,13]])
print(A)
Data = A - A.mean(0)
Data.round(2)
U, S, Vt = svd(Data)

"""

'\nsvd = np.linalg.svd\nA = array([[1, 2,7,11], [3, 4,8,12], [5, 6,9,13]])\nprint(A)\nData = A - A.mean(0)\nData.round(2)\nU, S, Vt = svd(Data)\n\n'